In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monthly-food-price-estimates/WLD_RTP_details_2023-10-02.csv
/kaggle/input/monthly-food-price-estimates/ddi-documentation-english_microdata-4483 (1).pdf
/kaggle/input/monthly-food-price-estimates/ddi-documentation-english_microdata-4509.pdf
/kaggle/input/monthly-food-price-estimates/WLD_RTFP_country_2023-10-02.csv
/kaggle/input/spark4ai-prediction/soil_dataset/sample_submission.csv
/kaggle/input/spark4ai-prediction/soil_dataset/train.csv
/kaggle/input/spark4ai-prediction/soil_dataset/test.csv


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df=pd.read_csv('/kaggle/input/spark4ai-prediction/soil_dataset/train.csv')
df_part1 = df.iloc[:, :23]
df_part2=df.iloc[:,23:]
df_part1.drop('Id',axis=1)
fd=df.head(101)
fx=pd.read_csv('/kaggle/input/spark4ai-prediction/soil_dataset/test.csv')
fx_part1 = fx.iloc[:, :17]
fx_part2=fx.iloc[:,17:]
fx_part1.columns
fx_part2





,350 nm,351 nm,352 nm,353 nm,354 nm,355 nm,356 nm,357 nm,358 nm,359 nm,...,2491 nm,2492 nm,2493 nm,2494 nm,2495 nm,2496 nm,2497 nm,2498 nm,2499 nm,2500 nm
0,0.232837,0.230487,0.228930,0.226657,0.225143,0.223977,0.220347,0.216707,0.213797,0.211987,...,0.390053,0.390647,0.391547,0.391620,0.391353,0.391007,0.390543,0.390117,0.389720,0.389410
1,0.295127,0.292557,0.290323,0.292620,0.291633,0.287363,0.285787,0.284977,0.284450,0.282020,...,0.394680,0.395323,0.396040,0.395467,0.394440,0.393930,0.393730,0.393800,0.394157,0.395083
2,0.281953,0.282010,0.281413,0.276690,0.275793,0.278263,0.275530,0.273700,0.274207,0.273223,...,0.396113,0.395733,0.395470,0.394950,0.394390,0.394030,0.393823,0.393937,0.394363,0.395190
3,0.258350,0.255567,0.251830,0.250390,0.248203,0.245277,0.243833,0.241513,0.237817,0.236643,...,0.446693,0.446783,0.447223,0.447357,0.447360,0.447080,0.446593,0.446193,0.445857,0.445790
4,0.220597,0.218647,0.216747,0.215900,0.214470,0.212477,0.211497,0.209313,0.205440,0.203700,...,0.349563,0.348673,0.348093,0.348110,0.348467,0.348823,0.349217,0.349270,0.348963,0.349193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.206923,0.205563,0.202637,0.199453,0.195797,0.192113,0.190050,0.189487,0.190157,0.186443,...,0.316780,0.315620,0.314637,0.314143,0.313890,0.313427,0.312940,0.313147,0.314017,0.315663
97,0.188527,0.184767,0.181907,0.182850,0.181467,0.177543,0.174700,0.173023,0.172453,0.168207,...,0.280483,0.279843,0.279640,0.279717,0.279877,0.279400,0.278517,0.277797,0.277193,0.277050
98,0.197503,0.194810,0.192490,0.192347,0.189500,0.184500,0.183987,0.182870,0.179777,0.175607,...,0.294737,0.294613,0.294897,0.295330,0.295707,0.295100,0.293843,0.293060,0.292703,0.292727
99,0.211407,0.211403,0.211033,0.206360,0.202050,0.198917,0.197477,0.195073,0.190940,0.187263,...,0.297890,0.297577,0.297727,0.298143,0.298627,0.298403,0.297723,0.297373,0.297323,0.297500


In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
model=RandomForestRegressor()
scaler_x = StandardScaler()
scaler_y = StandardScaler()

Y=df_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']]

X=df_part1.drop(['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F','Id'],axis=1)
X_s=scaler_x.fit_transform(X)
Y_s=scaler_y.fit_transform(Y)
X_t=fx_part1.drop(['Id'],axis=1)



X_train, X_val,Y_train, y_val = train_test_split(X_s, Y_s, test_size=0.401, random_state=42)


X_test=scaler_x.fit_transform(X_t)
model.fit(X_train,Y_train)
y_pred= model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val,y_pred))
r2=r2_score(y_val,y_pred)
print(f"Root Mean Squared Error (RMSE): {rmse}")
y=model.predict(X_test)
y_pred_original_scale = scaler_y.inverse_transform(y)
y_pred2=[]
for i in range(101):
    a=0 
    for j in range(6):
        a+=y_pred_original_scale [i][j]
    y_pred2.append(a) 
predict4=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred2})




Root Mean Squared Error (RMSE): 0.796123614946725


In [4]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
model = xgb.XGBRegressor()
model.fit(X_train, Y_train)
y_pred = model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
print(f"Mean Absolute Error: {mae}")
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")


Mean Absolute Error: 0.5439419544322067
Root Mean Squared Error (RMSE): 0.8447767376837794


In [5]:
X1=df_part2
model=RandomForestRegressor()

Y1=df_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']] 
X1_s=scaler_x.fit_transform(X1)
Y1_s=scaler_y.fit_transform(Y1)
X1_train, X1_val,Y1_train, y1_val = train_test_split(X1_s, Y1_s, test_size=0.401, random_state=42)
model.fit(X1_train, Y1_train)
y_pred = model.predict(X1_val)
X_t1=fx_part2
X_test1=scaler_x.fit_transform(X_t1)

mae = mean_absolute_error(y1_val, y_pred)
print(f"Mean Absolute Error: {mae}")
rmse = np.sqrt(mean_squared_error(y1_val, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")
y1=model.predict(X_test1)
y_pred_original_scale = scaler_y.inverse_transform(y1)
y_pred3=[]
for i in range(101):
    a=0 
    for j in range(6):
        a+=y_pred_original_scale [i][j]
    y_pred3.append(a) 
predict5=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred3})
predict5


Mean Absolute Error: 0.6033413229942798
Root Mean Squared Error (RMSE): 0.8883333738381539


,Id,Predicted
0,1,108.717259
1,2,108.462458
2,3,108.503704
3,4,107.681350
4,5,108.897438
...,...,...
96,97,108.160990
97,98,108.441566
98,99,108.434338
99,100,108.891718


In [6]:
model = xgb.XGBRegressor()
model.fit(X1_train, Y1_train)
y_pred = model.predict(X1_val)
mae = mean_absolute_error(y1_val, y_pred)
print(f"Mean Absolute Error: {mae}")
rmse = np.sqrt(mean_squared_error(y1_val, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")





Mean Absolute Error: 0.6532836395425036
Root Mean Squared Error (RMSE): 0.9748255203639553


In [7]:
bias_1 = 0.6
bias_2 = 0.4
fused_data = (bias_1*predict4 + bias_2 * predict5)/(bias_1+bias_2)
fused_data['Id']=fx['Id']
fused_data






,Id,Predicted
0,1,108.640129
1,2,108.707610
2,3,108.897812
3,4,108.171218
4,5,108.732382
...,...,...
96,97,107.977039
97,98,108.050936
98,99,107.889878
99,100,108.111569


In [8]:
csv_file_path = ' my_dataframe.csv' 
fused_data.to_csv(csv_file_path, index=False)